In [1]:
import pandas as pd
import pathlib
from recipe_preprocessor import import_recipes
from sentence_transformers import SentenceTransformer
from annoy import AnnoyIndex

Prepare model.

In [2]:
transformer_model: str = "paraphrase-MiniLM-L6-v2"

model: SentenceTransformer = SentenceTransformer(transformer_model)

Load data.

In [3]:
recipe_path = pathlib.Path("./data/recipes.csv")
recipes: pd.DataFrame = import_recipes("./data/recipes.csv")

In [70]:
ingredients = recipes["RecipeIngredientParts"].values

In [71]:
# main = set()
# for ing in ingredients:
#     main.update(ing)

In [77]:
macronutrient_cols = ["Calories", "FatContent", "SaturatedFatContent", "CholesterolContent", "SodiumContent", "CarbohydrateContent", "FiberContent", "SugarContent", "ProteinContent"]

macro_grouper = lambda row: [row[col] for col in macronutrient_cols]

recipes["Macros"] = recipes.apply(macro_grouper, axis=1)

In [80]:
len(recipes["Macros"][0])

9

In [82]:
embedding_indexes = zip(recipes["Macros"].index, recipes["Macros"])
vec_size = 9

index = AnnoyIndex(vec_size, "angular")

for embed in embedding_indexes:
    index.add_item(embed[0], embed[1])
    
index.build(10)
index.save(f"testIndexMacros.ann")



True

In [84]:
index = AnnoyIndex(vec_size, 'angular')
index.load('testIndexMacros.ann')

True

In [4]:
# recipes["Macros"].iloc[100]
recipes.iloc

In [92]:
testEmbedding = [55., 2., 0., 0., 7., 8., 1., 5., 1.5]
vectors = [recipes["Name"].loc[i] for i in index.get_nns_by_vector(testEmbedding, 10)]

In [93]:
vectors

['Korean Sesame Seed Cookies',
 'Roasted Cherry or Grape Tomatoes',
 'Portuguese Beef and Onions (Bifes De Cebolada)',
 "Julie's Rhubarb Bars or Crisp",
 'Peanut Butter, Banana, Chocolate Chip Brownies (Vegan or Not)',
 'Roasted Tomato, Pepper, and Red Onion Soup',
 'Raw Apple Crumble (No Bake)',
 'Baked Vegetable Ratatouille',
 'Maple-Pecan Granola',
 'Siena Cake - Panforte de Siena']

Create embeddings.

In [46]:
recipes["RecipeIngredientString"] = recipes["RecipeIngredientParts"].str.join(" ")

In [48]:
column_to_embed = "RecipeIngredientString"

## Generate embeddings and store in Annoy index

In [50]:
embeddings = model.encode(recipes[column_to_embed ].values)
embedding_indexes = zip(recipes[column_to_embed ].index, embeddings)
vec_size = 384

index = AnnoyIndex(vec_size, "angular")

for embed in embedding_indexes:
    index.add_item(embed[0], embed[1])
    
index.build(10)
index.save(f"testIndex{column_to_embed}.ann")

True

In [2]:
recipes

NameError: name 'recipes' is not defined

## Query Annoy index

Load index to memory

In [51]:
index = AnnoyIndex(vec_size, 'angular')
index.load('testIndex.ann')

True

Generate query embedding and execute query

In [52]:

testEmbedding = model.encode("sugar flour egg")

vectors = [recipes["Name"].loc[i] for i in index.get_nns_by_vector(testEmbedding, 10)]

In [53]:
vectors

['Oven Baked Eggs',
 'Whole Wheat Buttermilk Pancakes (With Wheat Germ)',
 'Buttermilk Wheat Pancakes',
 'Whole Wheat Buttermilk Pancakes',
 'Peanut Butter Eggs',
 'Rice Flour Pancakes',
 'Flourless Peanut Butter Pancakes',
 'Easy Cornmeal Buttermilk Pancakes',
 'Recipe: Homemade Buttermilk Pancakes',
 'Healthy Low-Fat Whole Wheat Buttermilk Pancakes']